# Dataset Cleaning

*Image Deleter and Relabeler by [Zach Caceres](http://zachcaceres.com/now/) in collaboration with Jason Patnick, Duplicate Finder by Francisco Ingham*

In this notebook we will show you how to take advantage of fastai widgets to clean your dataset! We will delete images that do not correspond, relabel images with incorrect labels and delete duplicates. For this, we will use the CIFAR10 dataset but you can use it in your own custom dataset you built with the [google images dataset](https://github.com/fpingham/google-images-dataset) notebook.

# Training your first model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
path = untar_data(URLs.CIFAR)

We will first train a model since it will suggest us which are the images that are most likely to be mislabelled or not belong to our dataset. We will also use the weights of the pretrained model to find similar images that might be duplicates.

In [4]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [5]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [6]:
# learn.fit_one_cycle(4)

In [7]:
# learn.save('stage-1');

In [8]:
learn.load('stage-1');

## Cleaning your dataset

In [9]:
from fastai.widgets import *
import torch

We will create an `ImageDataBunch` with all the images in the training set since the widget will only use the images in the training set.

In [10]:
# We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
src_del = (ImageItemList.from_folder(path)
                           .split_by_folder()
                           .label_from_folder())

items = ImageItemList([*src_del.train.x.items, *src_del.valid.x.items])
cats = CategoryList([*src_del.train.y.items, *src_del.valid.y.items], data.classes)

ll = LabelList(items, cats)

db = LabelLists(path, ll, None).databunch()

To start, we will sort the indices of our images by the highest loss 
images since this suggests that the image might be mislabeled or just not belong to the dataset.

In [11]:
learn_rel = create_cnn(db, models.resnet34, metrics=error_rate)
learn_rel.load('stage-1')

Learner(data=ImageDataBunch;
Train: LabelList
y: CategoryList (60000 items)
[Category dog, Category dog, Category dog, Category dog, Category dog]...
Path: .
x: ImageItemList (60000 items)
[Image (3, 32, 32), Image (3, 32, 32), Image (3, 32, 32), Image (3, 32, 32), Image (3, 32, 32)]...
Path: .;
Valid: None;
Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [12]:
ds, idxs = DatasetFormatter().from_toplosses(learn_rel)

Now we will use the widget to delete or move images. Flag photos for deletion by clicking 'Delete' or move them by using the dropdown menu. Then click 'Next Batch' to delete flagged photos and keep the rest in that row. `ImageCleaner` will show you a new row of images until there are no more to show. 

When you change your dataset, `ImageCleaner` will save the new dataset in a 'cleaned.csv' file in the same file where you have your notebook. 

In [13]:
ImageCleaner(ds, idxs, path)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

You can also find duplicates in your dataset and delete them! First we need to load our data from the csv and create a new `.Learner` object. 

In [17]:
df = pd.read_csv('cleaned.csv', header='infer')

In [18]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
src_dup = (ImageItemList.from_csv(path, 'cleaned.csv')
                        .random_split_by_pct()
                        .label_from_df())

items = ImageItemList([*src_dup.train.x.items, *src_dup.valid.x.items])
cats = CategoryList([*src_dup.train.y.items, *src_dup.valid.y.items], data.classes)

ll = LabelList(items, cats)

db = LabelLists(path, ll, None).databunch()

In [19]:
learn_dup = create_cnn(db, models.resnet34, metrics=error_rate)
learn_dup.load('stage-1');

We will first get the indexes for the most similar images and ImageCleaner will use them to find the potential duplicates. This step is quite memory intensive, so you might want to do monitor your RAM to see if your machine can handle it. It also takes quite a lot of time, if you want to make it faster you can edit the `pool_dim` parameter to make the feature vectors smaller (notice that the improvement in speed is exponential since, for example, decreasing dimension from 4 to 3 decreases the pooling layer's output from 4x4 (16) to 3x3 (9).

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn_dup, pool_dim=4)

Getting activations...


Computing similarities...


Take a look at the images in pairs and delete the ones you don't want to see anymore, until you feel that images don't look alike anymore. Remember that if you want to rerun the widget you need to recreate the `ImageDataBunch` object, loading the data from `cleaned.csv`.

In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

Turns out there is quite a number of duplicates in CIFAR!

## Train with new dataset

Now you are ready to work in your real training with a clean dataset! To use the new dataset you must load the labels and files from csv into your ImageDataBunch object.

In [86]:
np.random.seed(42)
data = ImageDataBunch.from_csv(path, folder=".", valid_pct=0.2, csv_labels='cleaned.csv',
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [87]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [45]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate
1,2.697477,1.928366,0.726316
2,2.147982,1.205164,0.421053
3,1.731222,1.010826,0.294737
4,1.483092,0.964751,0.326316
